<a href="https://colab.research.google.com/github/shirlymelendeza/Proyecto-final/blob/main/TRABAJOFINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo final


In [ ]:
import os
import platform
import random as rnd
import pandas as pd
import time
import datetime
import logging
import tqdm

In [ ]:
def GenerarNombre(Nombres: list, Apellidos: list) -> str:
    Nombre = rnd.choice(Nombres)
    Apellido = rnd.choice(Apellidos)
    return f'{Nombre.upper()}_{Apellido.upper()}'

def GenerearSemestre() -> int:
    semestre = [1] * 140
    semestre.extend([2] * 130)
    semestre.extend([3] * 120)
    semestre.extend([4] * 110)
    semestre.extend([5] * 100)
    semestre.extend([6] * 100)
    semestre.extend([7] * 90)
    semestre.extend([8] * 80)
    semestre.extend([9] * 70)
    semestre.extend([10] * 60)

    rnd.shuffle(semestre)
    return semestre



In [ ]:
# Region input data
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving ApellidosArgentina.csv to ApellidosArgentina (1).csv
Saving NombresArgentina.csv to NombresArgentina (1).csv
User uploaded file "ApellidosArgentina (1).csv" with length 8703778 bytes
User uploaded file "NombresArgentina (1).csv" with length 881357 bytes


In [ ]:
# Region input data
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Materias.xlsx to Materias (1).xlsx
User uploaded file "Materias (1).xlsx" with length 11873 bytes


In [ ]:
RutaNombres = '/content/NombresArgentina.csv'
RutaApellidos = '/content/ApellidosArgentina.csv'

dfNombres = pd.read_csv(RutaNombres, encoding='ISO-8859-1')
Nombres = dfNombres['name'].tolist()
for i in tqdm.trange(len(Nombres)):
    if ' ' in Nombres[i]:
        Nombres[i]=Nombres[i].replace(' ', '_')

dfApellidos = pd.read_csv(RutaApellidos, encoding='ISO-8859-1')
Apellidos = dfApellidos['lastname'].tolist()
for i in tqdm.trange(len(Apellidos)):
    if ' ' in Apellidos[i]:
        Apellidos[i]=Apellidos[i].replace(' ', '_')

df1 = pd.DataFrame(columns=['Nombre&Apellido'])
for i in tqdm.trange(1000):
    vector = []
    nombre = GenerarNombre(Nombres, Apellidos)
    vector = [nombre]
    df1.loc[len(df1)] = vector

100%|██████████| 1000/1000 [00:01<00:00, 794.67it/s]


In [ ]:
# Region creación semestres

semestres_generados = GenerearSemestre()
df2 = pd.DataFrame({'SEMESTRE': semestres_generados})

In [ ]:
# Region Data Frame con Nombres, apellido y semestre
Nombres= df1.join(df2, lsuffix='_df1', rsuffix='_df2')
Nombres

,Nombre&Apellido,SEMESTRE
0,ANTHONY_STEVEN_CERCADO,4
1,MICAELA_BELEN_BUCCONI,8
2,CONRADO_JOSE_LITARDO,9
3,EUGENIO_ENRIQUE_VAIMBERG,2
4,ASAHEL_DE_SANLIBORIO,2
...,...,...
995,ISAAC_ALBERTO_MICHELON,10
996,LEMUEL_NOTTE,2
997,URI_PREVETH,3
998,BRIAN_ANTONIO_GIRA,2


In [ ]:
#Región dataframe base de datos inicial
Materias = pd.read_excel('Materias.xlsx')
Materias

FileNotFoundError: [Errno 2] No such file or directory: 'Materias.xlsx'

In [ ]:
# Funciones fundamentales del programa

#Dependiendo del valor que se presente le asigna una cantidad máxima de cupos por grupo dependiendo del semestre en el que está
def condiciones_grupo(valor):
    if valor in [1, 2, 3]:
        return 30
    elif valor in [4, 5, 6]:
        return 25
    elif valor in [7, 8, 9]:
        return 20
    elif valor == 10:
        return 10
    else:
        return None

#Dependiendo del valor que esté arroja la cantidad de estudiantes por materia dependiendo del semestre en el que está
def condiciones_materia(valor):
    if valor == 1:
        return 140
    elif valor == 2:
        return 130
    elif valor == 3:
        return 120
    elif valor == 4:
        return 110
    elif valor in [5, 6]:
        return 100
    elif valor == 7:
        return 90
    elif valor == 8:
        return 80
    elif valor == 9:
        return 70
    elif valor == 10:
        return 60
    else:
        return None

#Arroja la cantidad máxima de grpos por materia
def cantidad_grupos(row):
  cociente = row['CUPOS TOTALES']//row['CUPOS POR GRUPO'] # Divide la cantidad de estudiantes entre los cupos por grupo para tener la cantidad mínima de grupos
  residuo = row['CUPOS TOTALES']%row['CUPOS POR GRUPO'] # Arroja el residuo de la división anterior para saber si la cantidad mínima de grupos es insuficiente para matricular a todos los estudiantes
  if residuo != 0:
    return cociente + 1 # Si el residuo es diferente de 0 se crea otro grupo para que todos los estudiantes puedan ser matriculados en un grupo
  else:
    return cociente # Si el residuo es 0 quiere decir que la cantidad mínima de grupos alcanza a matricular a todos los estudiantes

# Dependiendo del valor que esté arroja la cantidad horas trabajo docente dependiendo de la cantidad de creditos de la materia
def horas_docente(valor):
    if valor == 1:
        return 16
    elif valor == 2:
        return 32
    elif valor == 3:
        return 64
    elif valor == 4:
        return 96
    else:
        return None

# Dependiendo del valor que esté arroja la cantidad horas trabajo independiente dependiendo de la cantidad de creditos de la materia
def horas_independiente(valor):
    if valor == 1:
        return 32
    elif valor == 2:
        return 64
    elif valor == 3:
        return 80
    elif valor == 4:
        return 120
    else:
        return None

In [ ]:
#Se crea una nueva columna al dataframe inicial aplicando a la columna de semestre la función que arroja la cantidad de estudiantes por materia
Materias['CUPOS TOTALES'] = Materias['SEMESTRE'].apply(condiciones_materia)
Materias

,MATERIA,SEMESTRE,CREDITOS,CUPOS TOTALES
0,Inglés 1,1,1,140
1,Introducción a la ingeniería industrial,1,1,140
2,Lectoescritura,1,3,140
3,Vivamos la universidad,1,1,140
4,Cálculo diferencial,1,3,140
5,Álgebra y trigonometría,1,3,140
6,Geometría vectorial,1,3,140
7,Inglés 2,2,1,130
8,Gestión de las organizaciones,2,3,130
9,Habilidades gerenciales,2,3,130


In [ ]:
#Se crea una nueva columna al dataframe inicial aplicando a la columna de semestre la función que arroja la cantidad de cupos por grupo por materia
Materias['CUPOS POR GRUPO'] = Materias['SEMESTRE'].apply(condiciones_grupo)
Materias

,MATERIA,SEMESTRE,CREDITOS,CUPOS TOTALES,CUPOS POR GRUPO
0,Inglés 1,1,1,140,30
1,Introducción a la ingeniería industrial,1,1,140,30
2,Lectoescritura,1,3,140,30
3,Vivamos la universidad,1,1,140,30
4,Cálculo diferencial,1,3,140,30
5,Álgebra y trigonometría,1,3,140,30
6,Geometría vectorial,1,3,140,30
7,Inglés 2,2,1,130,30
8,Gestión de las organizaciones,2,3,130,30
9,Habilidades gerenciales,2,3,130,30


In [ ]:
#Se crea una nueva columna al dataframe inicial aplicando al dataframe completo la función que arroja la cantidad máxima de grupos por materia
Materias['Cantidad de grupos'] = Materias.apply(lambda row: cantidad_grupos(row), axis = 1)
Materias

,MATERIA,SEMESTRE,CREDITOS,CUPOS TOTALES,CUPOS POR GRUPO,Cantidad de grupos
0,Inglés 1,1,1,140,30,5
1,Introducción a la ingeniería industrial,1,1,140,30,5
2,Lectoescritura,1,3,140,30,5
3,Vivamos la universidad,1,1,140,30,5
4,Cálculo diferencial,1,3,140,30,5
5,Álgebra y trigonometría,1,3,140,30,5
6,Geometría vectorial,1,3,140,30,5
7,Inglés 2,2,1,130,30,5
8,Gestión de las organizaciones,2,3,130,30,5
9,Habilidades gerenciales,2,3,130,30,5


In [ ]:
#Se crea una nueva columna al dataframe inicial aplicando a la columna de creditos la función que arroja la cantidad de horas trabajo docente
Materias['HTD'] = Materias['CREDITOS'].apply(horas_docente)
Materias

,MATERIA,SEMESTRE,CREDITOS,CUPOS TOTALES,CUPOS POR GRUPO,Cantidad de grupos,HTD,HTI
0,Inglés 1,1,1,140,30,5,16.0,32.0
1,Introducción a la ingeniería industrial,1,1,140,30,5,16.0,32.0
2,Lectoescritura,1,3,140,30,5,64.0,80.0
3,Vivamos la universidad,1,1,140,30,5,16.0,32.0
4,Cálculo diferencial,1,3,140,30,5,64.0,80.0
5,Álgebra y trigonometría,1,3,140,30,5,64.0,80.0
6,Geometría vectorial,1,3,140,30,5,64.0,80.0
7,Inglés 2,2,1,130,30,5,16.0,32.0
8,Gestión de las organizaciones,2,3,130,30,5,64.0,80.0
9,Habilidades gerenciales,2,3,130,30,5,64.0,80.0


In [ ]:
#Se crea una nueva columna al dataframe inicial aplicando a la columna de creditos la función que arroja la cantidad de horas trabajo independiente
Materias['HTI'] = Materias['CREDITOS'].apply(horas_independiente)
Materias

,MATERIA,SEMESTRE,CREDITOS,CUPOS TOTALES,CUPOS POR GRUPO,Cantidad de grupos,HTD,HTI
0,Inglés 1,1,1,140,30,5,16.0,32.0
1,Introducción a la ingeniería industrial,1,1,140,30,5,16.0,32.0
2,Lectoescritura,1,3,140,30,5,64.0,80.0
3,Vivamos la universidad,1,1,140,30,5,16.0,32.0
4,Cálculo diferencial,1,3,140,30,5,64.0,80.0
5,Álgebra y trigonometría,1,3,140,30,5,64.0,80.0
6,Geometría vectorial,1,3,140,30,5,64.0,80.0
7,Inglés 2,2,1,130,30,5,16.0,32.0
8,Gestión de las organizaciones,2,3,130,30,5,64.0,80.0
9,Habilidades gerenciales,2,3,130,30,5,64.0,80.0


In [ ]:
#Creamos un dataframe nuevo uniendo el dataframe que tiene toda la info de las materias con el dataframe que contiene los nombres
Completo = pd.merge(Materias, Nombres, on='SEMESTRE') #Se unen a partir del criterio/columna que ambos comparten: Semestre
Completo

,MATERIA,SEMESTRE,CREDITOS,CUPOS TOTALES,CUPOS POR GRUPO,Cantidad de grupos,HTD,HTI,Nombre&Apellido
0,Inglés 1,1,1,140,30,5,16.0,32.0,ISAAK_PLANO
1,Inglés 1,1,1,140,30,5,16.0,32.0,MARCIA_ISABEL_BELLAGAMBA
2,Inglés 1,1,1,140,30,5,16.0,32.0,PROFIRA_GEVARA
3,Inglés 1,1,1,140,30,5,16.0,32.0,STEPHANIE_JANE_ZENTIL
4,Inglés 1,1,1,140,30,5,16.0,32.0,ESTEBAN_RAUL_LEONI
...,...,...,...,...,...,...,...,...,...
5955,Práctica profesional,10,12,60,10,6,NaN,NaN,ELENA_VIRGINIA_ZENARDO
5956,Práctica profesional,10,12,60,10,6,NaN,NaN,GABRIEL_NICOLAS_CASTELLARIN
5957,Práctica profesional,10,12,60,10,6,NaN,NaN,ASIF_BORGNA
5958,Práctica profesional,10,12,60,10,6,NaN,NaN,ISAURO_MASDEU


In [ ]:
# Creamos una columna nueva que asigna a que grupo de cada materia pertenece el estudiante

Completo['GRUPO'] = Completo.apply(lambda row: ((row.name // row['CUPOS POR GRUPO']) % row['Cantidad de grupos']) + 1, axis=1)

# Se utiliza lambda para escribir de forma compacta la función ya que es una expresión corta.
# Divide (tomando solo la parte entera) el índice de la fila entre los cupos por grupo y le calcula el residuo de dividir el resultado entre la cantidad de grupos
# El número que nos arroja le sumamos 1 ya que los indices en python empiezan en 0 pero los grupos empiezan desde 1.

Completo

,MATERIA,SEMESTRE,CREDITOS,CUPOS TOTALES,CUPOS POR GRUPO,Cantidad de grupos,HTD,HTI,Nombre&Apellido,GRUPO
0,Inglés 1,1,1,140,30,5,16.0,32.0,ISAAK_PLANO,1
1,Inglés 1,1,1,140,30,5,16.0,32.0,MARCIA_ISABEL_BELLAGAMBA,1
2,Inglés 1,1,1,140,30,5,16.0,32.0,PROFIRA_GEVARA,1
3,Inglés 1,1,1,140,30,5,16.0,32.0,STEPHANIE_JANE_ZENTIL,1
4,Inglés 1,1,1,140,30,5,16.0,32.0,ESTEBAN_RAUL_LEONI,1
...,...,...,...,...,...,...,...,...,...,...
5955,Práctica profesional,10,12,60,10,6,NaN,NaN,ELENA_VIRGINIA_ZENARDO,2
5956,Práctica profesional,10,12,60,10,6,NaN,NaN,GABRIEL_NICOLAS_CASTELLARIN,2
5957,Práctica profesional,10,12,60,10,6,NaN,NaN,ASIF_BORGNA,2
5958,Práctica profesional,10,12,60,10,6,NaN,NaN,ISAURO_MASDEU,2


In [ ]:
Completo['FECHA DE CREACIÓN'] = datetime.datetime.now().strftime('%Y-%m-%d')
Completo

,MATERIA,SEMESTRE,CREDITOS,CUPOS TOTALES,CUPOS POR GRUPO,Cantidad de grupos,HTD,HTI,Nombre&Apellido,GRUPO,FECHA DE CREACIÓN
0,Inglés 1,1,1,140,30,5,16.0,32.0,ISAAK_PLANO,1,2024-05-29
1,Inglés 1,1,1,140,30,5,16.0,32.0,MARCIA_ISABEL_BELLAGAMBA,1,2024-05-29
2,Inglés 1,1,1,140,30,5,16.0,32.0,PROFIRA_GEVARA,1,2024-05-29
3,Inglés 1,1,1,140,30,5,16.0,32.0,STEPHANIE_JANE_ZENTIL,1,2024-05-29
4,Inglés 1,1,1,140,30,5,16.0,32.0,ESTEBAN_RAUL_LEONI,1,2024-05-29
...,...,...,...,...,...,...,...,...,...,...,...
5955,Práctica profesional,10,12,60,10,6,NaN,NaN,ELENA_VIRGINIA_ZENARDO,2,2024-05-29
5956,Práctica profesional,10,12,60,10,6,NaN,NaN,GABRIEL_NICOLAS_CASTELLARIN,2,2024-05-29
5957,Práctica profesional,10,12,60,10,6,NaN,NaN,ASIF_BORGNA,2,2024-05-29
5958,Práctica profesional,10,12,60,10,6,NaN,NaN,ISAURO_MASDEU,2,2024-05-29


In [ ]:
def generar_codigo(row):
    try:
        Materia = row['MATERIA'].split() # Separa cada palabra de la columna y lo convierte en una lista de strings
        codigo = ''.join([Materia[0][:2].upper(), Materia[1][0].upper(), str(row['SEMESTRE']), str(row['GRUPO'])]) #Toma las primeras dos letras de la primera palabra del nombre de la materia, la primera letra de la segunda palabra, el número del semestre y el grupo
        return codigo
    except IndexError: # En caso de que no pueda tomar una posición de la columna o se presente algún error arroja "None"
        codigo_alternativo = ''
        if Materia:
          codigo_alternativo = ''.join([Materia[0][:3].upper(), str(row['SEMESTRE']), str(row['GRUPO'])])
        return codigo_alternativo or None


Completo['CODIGO'] = Completo.apply(generar_codigo, axis=1)
Completo

,MATERIA,SEMESTRE,CREDITOS,CUPOS TOTALES,CUPOS POR GRUPO,Cantidad de grupos,HTD,HTI,Nombre&Apellido,GRUPO,FECHA DE CREACIÓN,CODIGO
0,Inglés 1,1,1,140,30,5,16.0,32.0,ISAAK_PLANO,1,2024-05-29,IN111
1,Inglés 1,1,1,140,30,5,16.0,32.0,MARCIA_ISABEL_BELLAGAMBA,1,2024-05-29,IN111
2,Inglés 1,1,1,140,30,5,16.0,32.0,PROFIRA_GEVARA,1,2024-05-29,IN111
3,Inglés 1,1,1,140,30,5,16.0,32.0,STEPHANIE_JANE_ZENTIL,1,2024-05-29,IN111
4,Inglés 1,1,1,140,30,5,16.0,32.0,ESTEBAN_RAUL_LEONI,1,2024-05-29,IN111
...,...,...,...,...,...,...,...,...,...,...,...,...
5955,Práctica profesional,10,12,60,10,6,NaN,NaN,ELENA_VIRGINIA_ZENARDO,2,2024-05-29,PRP102
5956,Práctica profesional,10,12,60,10,6,NaN,NaN,GABRIEL_NICOLAS_CASTELLARIN,2,2024-05-29,PRP102
5957,Práctica profesional,10,12,60,10,6,NaN,NaN,ASIF_BORGNA,2,2024-05-29,PRP102
5958,Práctica profesional,10,12,60,10,6,NaN,NaN,ISAURO_MASDEU,2,2024-05-29,PRP102


In [ ]:

# Crea una carpeta general
carpeta_general = 'carpeta-general'
os.makedirs(carpeta_general, exist_ok=True)

# Agrupa el dataframe por la columna de las materias y el semestre
grupos_por_materia_y_semestre = Completo.groupby(['MATERIA', 'SEMESTRE'])

# Crea carpetas para cada semestre
for semestre, grupo in grupos_por_materia_y_semestre:
    carpeta_semestre = os.path.join(carpeta_general, str(semestre[1]))
    os.makedirs(carpeta_semestre, exist_ok=True)

    # Exporta los archivos correspondientes a cada semestre
    for nombre_materia, subgrupo in grupo.groupby('MATERIA'):
        # Quita los espacios y capitaliza el nombre de la materia
        nombre_materia_modificado = ''.join(word.capitalize() for word in nombre_materia.split())
        # Obtiene la cantidad de estudiantes y el código alfanumérico
        cantidad_estudiantes = subgrupo['CUPOS TOTALES'].iloc[0]
        codigo = subgrupo['CODIGO'].iloc[0]
        # Crea el nombre del archivo
        archivo_nombre = f'{nombre_materia_modificado}-{codigo}-{cantidad_estudiantes}.xlsx'
        subgrupo.to_excel(os.path.join(carpeta_semestre, archivo_nombre), index=False)